In [1]:
from numba import njit, prange
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

In [2]:
def visualize_solution_3d_by_t(u_analytical, u_asm, u_sm, x, y, t, t_ix, tau, h, n_solutions, title):
    t_colorscale = px.colors.sample_colorscale("Jet", y)
    fig = go.Figure()

    fig.add_scatter3d(x=x, y=np.ones_like(x) * y[0], z=u_analytical(x, y[0], t[t_ix]), legendgroup="analytical", name="Analytical", mode="lines", line=dict(color=t_colorscale[0], width=3))
    for i in range(int(len(y) / n_solutions), len(y), int(len(y) / n_solutions)):  # Adjust the step size for clarity
        fig.add_scatter3d(x=x, y=np.ones_like(x) * y[i], z=u_analytical(x, y[i], t[t_ix]), legendgroup="analytical", showlegend=False, mode="lines", line=dict(color=t_colorscale[i], width=3))


    fig.add_scatter3d(x=x, y=np.ones_like(x) * y[0], z=u_asm[t_ix, :, 0], legendgroup="asm", name="Adaptive step", mode="lines", line=dict(color="blue", width=5, dash="dot"), marker=dict(size=1))
    for i in range(int(len(y) / n_solutions), len(y), int(len(y) / n_solutions)):  # Adjust the step size for clarity
        fig.add_scatter3d(x=x, y=np.ones_like(x) * y[i], z=u_asm[t_ix, :, i], legendgroup="asm", showlegend=False, mode="lines", line=dict(color="blue", width=5, dash="dot"), marker=dict(size=1))

    fig.add_scatter3d(x=x, y=np.ones_like(x) * y[0], z=u_sm[t_ix, :, 0], legendgroup="sm", name="Splitting", mode="lines", line=dict(color="red", width=4, dash="longdash"), marker=dict(size=2))
    for i in range(int(len(y) / n_solutions), len(y), int(len(y) / n_solutions)):  # Adjust the step size for clarity
        fig.add_scatter3d(x=x, y=np.ones_like(x) * y[i], z=u_sm[t_ix, :, i], legendgroup="implicit", showlegend=False, mode="lines", line=dict(color="red", width=4, dash="longdash"), marker=dict(size=1))


    fig.update_scenes(xaxis_title_text='x',
                      yaxis_title_text='y',
                      zaxis_title_text=f'u({t[t_ix]:.2f}, x, y)')
    fig.update_layout(width=1080, height=720)

    camera = dict(
        eye=dict(x=-1.6, y=-1.6, z=1)
    )

    fig.update_layout(
        scene_camera=camera,
        title=f"$\\text{{{title}}}t = {t[t_ix]:.2f}, \\Delta y = \\Delta x = {h:.1e}, \\Delta t = {tau:.1e}$",
        title_x=0.5
    )
    return fig

In [3]:
def visualize_solution_3d_by_y(u_analytical, u_asm, u_sm, x, y, y_ix, t, tau, h, n_solutions, title):
    t_colorscale = px.colors.sample_colorscale("Jet", t)
    fig = go.Figure()

    fig.add_scatter3d(x=x, y=np.ones_like(x) * t[0], z=u_analytical(x, t[0], y[y_ix]), legendgroup="analytical", name="Analytical", mode="lines", line=dict(color=t_colorscale[0], width=3))
    for i in range(int(len(t) / n_solutions), len(t), int(len(t) / n_solutions)):  # Adjust the step size for clarity
        fig.add_scatter3d(x=x, y=np.ones_like(x) * t[i], z=u_analytical(x, y[y_ix], t[i]), legendgroup="analytical", showlegend=False, mode="lines", line=dict(color=t_colorscale[i], width=3))


    fig.add_scatter3d(x=x, y=np.ones_like(x) * t[0], z=u_asm[0, :, y_ix], legendgroup="asm", name="Adaptive step", mode="lines", line=dict(color="blue", width=5, dash="dot"), marker=dict(size=1))
    for i in range(int(len(t) / n_solutions), len(t), int(len(t) / n_solutions)):  # Adjust the step size for clarity
        fig.add_scatter3d(x=x, y=np.ones_like(x) * t[i], z=u_asm[i, :, y_ix], legendgroup="asm", showlegend=False, mode="lines", line=dict(color="blue", width=5, dash="dot"), marker=dict(size=1))

    fig.add_scatter3d(x=x, y=np.ones_like(x) * t[0], z=u_sm[0, :, y_ix], legendgroup="sm", name="Splitting", mode="lines", line=dict(color="red", width=4, dash="longdash"), marker=dict(size=2))
    for i in range(int(len(t) / n_solutions), len(t), int(len(t) / n_solutions)):  # Adjust the step size for clarity
        fig.add_scatter3d(x=x, y=np.ones_like(x) * t[i], z=u_sm[i, :, y_ix], legendgroup="implicit", showlegend=False, mode="lines", line=dict(color="red", width=4, dash="longdash"), marker=dict(size=1))


    fig.update_scenes(xaxis_title_text='x',
                      yaxis_title_text='t',
                      zaxis_title_text=f'u(t, x, {y[y_ix]:.2f})')
    fig.update_layout(width=1080, height=720)

    camera = dict(
        eye=dict(x=-1.6, y=-1.6, z=1)
    )

    fig.update_layout(
        scene_camera=camera,
        title=f"$\\text{{{title}}}y = {y[y_ix]:.2f}, \\Delta y = \\Delta x = {h:.1e}, \\Delta t = {tau:.1e}$",
        title_x=0.5
    )
    return fig

In [4]:
# Thomas algorithm
@njit
def tdma(l: np.ndarray, m: np.ndarray, u: np.ndarray, d: np.ndarray) -> np.ndarray:
    n = len(d)
    alpha = np.empty(n)
    beta = np.empty(n)
    x = np.empty(n)

    alpha[0] = u[0] / m[0]
    beta[0] = d[0] / m[0]
    for i in range(1, n):
        alpha[i] = u[i] / (m[i] - l[i] * alpha[i - 1])
        beta[i] = (d[i] - l[i] * beta[i - 1]) / (m[i] - l[i] * alpha[i - 1])

    x[-1] = beta[-1]

    for i in range(-2, -n - 1, -1):
        x[i] = beta[i] - alpha[i] * x[i + 1]
    return x

In [5]:
@njit
def f(x, y, t):
    return x * y * np.cos(t)

def u_analytical(x: np.ndarray, y: np.ndarray, t: np.ndarray) -> np.ndarray:
    return x * y * np.sin(t)

def get_grid(m, n):
    T = 1
    t, tau = np.linspace(0, T, 2 * m + 1, retstep=True) # fractional steps
    x, h = np.linspace(0, 1, n + 1, retstep=True)
    y = np.linspace(0, 1, n + 1)
    return t, x, y, tau, h

def init_u(n, m, x, y, t):
    u = np.empty(shape=(2 * m + 1, n + 1, n + 1))  # matrix for numerical solution of u
    u[0, ...] = 0
    u[:, 0, :] = 0  # left boundary condition by x
    u[:, -1, :] = y * np.sin(t)[:, np.newaxis] # right boundary condition by x
    u[:, :, 0] = 0  # bottom boundary condition by y
    u[:, :, -1] = x * np.sin(t)[:, np.newaxis] # top boundary condition by y
    return u

In [6]:
N = 2 ** 5
M = 2 ** 12
t, x, y, tau, h = get_grid(M, N)

tau / h ** 2 < 1/4

np.True_

## Adaptive Step Method

In [7]:
u_asm = init_u(N, M, x, y, t)

In [8]:
@njit
def next_step_by_asm(u: np.ndarray, t: np.ndarray, x: np.ndarray, y: np.ndarray, prev_ix: int, h: float,
                                     tau: float):
    n = u.shape[1]
    a = np.full(n - 1, -1)
    a[0] = 0

    b = np.full(n - 1, 2 + 2 * h ** 2 / (3 * tau))

    c = np.full(n - 1, -1)
    c[-1] = 0
    j = prev_ix + 1
    for i in range(1, n-1):
        d = u[prev_ix, 1:-1, i + 1] + (2 * h ** 2 / 3 / tau - 2) * u[prev_ix, 1:-1, i] + u[prev_ix, 1:-1, i - 1] + h ** 2 / 3 * f(x[1:-1], y[i], t[j])
        d[-1] += y[i] * np.sin(t[j])
        u[j, 1:-1, i] = tdma(a, b, c, d)
    
    prev_ix = j
    j += 1
    for i in range(1, n-1):
        d = u[prev_ix, i + 1, 1:-1] + (2 * h ** 2 / 3 / tau - 2) * u[prev_ix, i, 1:-1] + u[prev_ix, i - 1, 1:-1] + h ** 2 / 3 * f(x[i], y[1:-1], t[prev_ix])
        d[-1] += x[i] * np.sin(t[j])
        u[j, i, 1:-1] = tdma(a, b, c, d)

In [9]:
for k in range(0, 2*M, 2):
    next_step_by_asm(u_asm, t, x, y, k, h, tau)

## Splitting method

In [10]:
u_sm = init_u(N, M, x, y, t)

In [11]:
@njit
def next_step_by_sm(u: np.ndarray, t: np.ndarray, x: np.ndarray, y: np.ndarray, prev_ix: int, h: float,
                     tau: float):
    n = u.shape[1]
    a = np.full(n - 1, -1)
    a[0] = 0

    b = np.full(n - 1, 2 + h ** 2 / (3 * tau))

    c = np.full(n - 1, -1)
    c[-1] = 0
    j = prev_ix + 1
    for i in range(1, n-1):
        d = h ** 2 / 3 / tau * u[prev_ix, 1:-1, i] + h ** 2 / 6 * f(x[1:-1], y[i], t[j])
        d[-1] += y[i] * np.sin(t[j])
        u[j, 1:-1, i] = tdma(a, b, c, d)

    prev_ix = j
    j += 1
    for i in range(1, n-1):
        d = h ** 2 / 3 / tau * u[prev_ix, i, 1:-1] + h ** 2 / 6 * f(x[i], y[1:-1], t[prev_ix])
        d[-1] += x[i] * np.sin(t[j])
        u[j, i, 1:-1] = tdma(a, b, c, d)

In [12]:
for k in range(0, 2*M, 2):
    next_step_by_sm(u_sm, t, x, y, k, h, tau)

In [13]:
fig = visualize_solution_3d_by_t(u_analytical, u_asm, u_sm, x, y, t, -1, tau, h, 30, "The numerical solution of two-dim heat equation ")
fig.write_image("3d_2dim_heat_eq_t_1.png", scale=6)

In [14]:
fig = visualize_solution_3d_by_t(u_analytical, u_asm, u_sm, x, y, t, len(t)//2, tau, h, 30, "The numerical solution of two-dim heat equation ")
fig.write_image(f"3d_2dim_heat_eq_half_t.png", scale=6)

In [15]:
fig = visualize_solution_3d_by_y(u_analytical, u_asm, u_sm, x, y, len(y)// 8, t, tau, h, 30, "The numerical solution of two-dim heat equation ")
fig.write_image(f"3d_2dim_heat_eq_eighth_y.png", scale=6)

In [16]:
N = 2 ** 4
M = 2 ** 6
t, x, y, tau, h = get_grid(M, N)

print(tau / h ** 2 < 1/4)

u_asm = init_u(N, M, x, y, t)

for k in range(0, 2*M, 2):
    next_step_by_asm(u_asm, t, x, y, k, h, tau)

u_sm = init_u(N, M, x, y, t)

for k in range(0, 2*M, 2):
    next_step_by_sm(u_sm, t, x, y, k, h, tau)

False


In [17]:
fig = go.Figure()

(fig.add_scatter(x=t, y=abs(u_asm[:, 5, 5] - u_analytical(x[5], y[5], t)), name="Adaptive Step")
 .add_scatter(x=t, y=abs(u_sm[:, 5, 5] - u_analytical(x[5], y[5], t)), name="Splitting")
 )

fig.update_layout(xaxis_title="t", yaxis_title="Absolute Error", title=f"$\\text{{Absolute error on the slice by }} y = {y[5]:.2f}, x = {x[5]:.2f}, \\Delta x = \\Delta y = {h:.2e}, \\Delta t = {tau:.2e}$", yaxis_tickformat=".2e", width=1280, height=720)
fig.write_image("abs_err_x_y.png", scale=6)

In [18]:
fig = go.Figure()

(fig.add_scatter(x=x, y=abs(u_asm[-1, :, 5] - u_analytical(x, y[5], t[-1])), name="Adaptive Step")
 .add_scatter(x=x, y=abs(u_sm[-1, :, 5] - u_analytical(x, y[5], t[-1])), name="Splitting")
 )

fig.update_layout(xaxis_title="x", yaxis_title="Absolute Error", title=f"$\\text{{Absolute error on the slice by }} t = {t[-1]:.2f}, y = {y[5]:.2f}, \\Delta x = \\Delta y = {h:.2e}, \\Delta t = {tau:.2e}$", yaxis_tickformat=".2e", width=1280, height=720)
fig.write_image("abs_err_t_y.png", scale=6)

In [19]:
N = 2 ** 6 # increased number of steps
M = 2 ** 6
t, x, y, tau, h = get_grid(M, N)

print(tau / h ** 2 < 1 / 4)

u_asm = init_u(N, M, x, y, t)

for k in range(0, 2 * M, 2):
    next_step_by_asm(u_asm, t, x, y, k, h, tau)

u_sm = init_u(N, M, x, y, t)

for k in range(0, 2 * M, 2):
    next_step_by_sm(u_sm, t, x, y, k, h, tau)
fig = go.Figure()

(fig.add_scatter(x=t, y=abs(u_asm[:, 5, 5] - u_analytical(x[5], y[5], t)), name="Adaptive Step")
 .add_scatter(x=t, y=abs(u_sm[:, 5, 5] - u_analytical(x[5], y[5], t)), name="Splitting")
 )

fig.update_layout(xaxis_title="t", yaxis_title="Absolute Error",
                  title=f"$\\text{{Absolute error on the slice by }} y = {y[5]:.2f}, x = {x[5]:.2f}, \\Delta x = \\Delta y = {h:.2e}, \\Delta t = {tau:.2e}$",
                  yaxis_tickformat=".2e", width=1280, height=720)
fig.write_image("abs_err_x_y_inc_n.png", scale=6)

False


In [20]:
fig = go.Figure()

(fig.add_scatter(x=x, y=abs(u_asm[-1, :, 5] - u_analytical(x, y[5], t[-1])), name="Adaptive Step")
 .add_scatter(x=x, y=abs(u_sm[-1, :, 5] - u_analytical(x, y[5], t[-1])), name="Splitting")
 )

fig.update_layout(xaxis_title="x", yaxis_title="Absolute Error", title=f"$\\text{{Absolute error on the slice by }} t = {t[-1]:.2f}, y = {y[5]:.2f}, \\Delta x = \\Delta y = {h:.2e}, \\Delta t = {tau:.2e}$", yaxis_tickformat=".2e", width=1280, height=720)
fig.write_image("abs_err_t_y_inc_n.png", scale=6)


In [21]:
N = 2 ** 6 
M = 2 ** 12 # increased number of steps
t, x, y, tau, h = get_grid(M, N)

print(tau / h ** 2 < 1 / 4)

u_asm = init_u(N, M, x, y, t)

for k in range(0, 2 * M, 2):
    next_step_by_asm(u_asm, t, x, y, k, h, tau)

u_sm = init_u(N, M, x, y, t)

for k in range(0, 2 * M, 2):
    next_step_by_sm(u_sm, t, x, y, k, h, tau)
fig = go.Figure()

(fig.add_scatter(x=t, y=abs(u_asm[:, 5, 5] - u_analytical(x[5], y[5], t)), name="Adaptive Step")
 .add_scatter(x=t, y=abs(u_sm[:, 5, 5] - u_analytical(x[5], y[5], t)), name="Splitting")
 )

fig.update_layout(xaxis_title="t", yaxis_title="Absolute Error",
                  title=f"$\\text{{Absolute error on the slice by }} y = {y[5]:.2f}, x = {x[5]:.2f}, \\Delta x = \\Delta y = {h:.2e}, \\Delta t = {tau:.2e}$",
                  yaxis_tickformat=".2e", width=1280, height=720)
fig.write_image("abs_err_x_y_inc_m.png", scale=6)


False


In [22]:
fig = go.Figure()

(fig.add_scatter(x=x, y=abs(u_asm[-1, :, 5] - u_analytical(x, y[5], t[-1])), name="Adaptive Step")
 .add_scatter(x=x, y=abs(u_sm[-1, :, 5] - u_analytical(x, y[5], t[-1])), name="Splitting")
 )

fig.update_layout(xaxis_title="x", yaxis_title="Absolute Error", title=f"$\\text{{Absolute error on the slice by }} t = {t[-1]:.2f}, y = {y[5]:.2f}, \\Delta x = \\Delta y = {h:.2e}, \\Delta t = {tau:.2e}$", yaxis_tickformat=".2e", width=1280, height=720)
fig.write_image("abs_err_t_y_inc_m.png", scale=6)